In [1]:
import re
import numpy as np
import pandas as pd
import pickle
import joblib
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report

# 사용 모델
from sklearn.ensemble import RandomForestClassifier
import lib.EncoderModel as mm

# 함수들 불러오기
import lib.functions as f

# 평가에 이용되는 함수 정의

In [2]:
# 1차 filter용 데이터셋 정의 -> (조문 + 조문제목) [embedding vector화 + 배치화]
class CustomDataset1(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # '조문'과 '법령명' 칼럼의 데이터를 안전하게 문자열로 변환하고 결합
        text1 = str(self.data.iloc[idx]['조문'])
        text2 = str(self.data.iloc[idx]['조문제목'])
        text_data = text1 + " // " + text2
        label = torch.tensor(self.data.iloc[idx]['사무판단'], dtype=torch.long)

        # 결합된 텍스트를 토크나이징
        tokenized_data = tokenizer([text_data], padding='max_length', max_length=512, truncation=True, return_tensors='pt')

        return {
            'input_ids': tokenized_data['input_ids'].squeeze(),
            'attention_mask': tokenized_data['attention_mask'].squeeze(),
            'labels': label
        }

In [3]:
# 2차 filter용 데이터셋 정의 -> (법령명 + 조문제목 + 수행주체) [embedding vector화 + 배치화]
class CustomDataset2(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # '조문'과 '법령명' 칼럼의 데이터를 안전하게 문자열로 변환하고 결합
        text1 = str(self.data.iloc[idx]['법령명'])
        text2 = str(self.data.iloc[idx]['조문제목'])
        text3 = str(self.data.iloc[idx]['수행주체(문자열)'])
        text_data = text1 + " // " + text2 + " // " + text3
        label = torch.tensor(self.data.iloc[idx]['사무유형(대분류)'], dtype=torch.long)

        # 결합된 텍스트를 토크나이징
        tokenized_data = tokenizer([text_data], padding='max_length', max_length=128, truncation=True, return_tensors='pt')

        return {
            'input_ids': tokenized_data['input_ids'].squeeze(),
            'attention_mask': tokenized_data['attention_mask'].squeeze(),
            'labels': label
        }

In [4]:
# 각 배치의 텍스트 길이를 맞추기
def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    labels = [item['labels'] for item in batch]

    # 패딩 적용
    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)

    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': torch.stack(labels)}


In [5]:
# 평가 결과 확인
def cal_result(preds, y):
    conf_matrix = confusion_matrix(y, preds)
    accuracy = accuracy_score(y, preds)
    precision = precision_score(y, preds)
    recall = recall_score(y, preds)
    f1 = f1_score(y, preds)
    
    # 결과 출력
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print("Confusion Matrix:")
    print(conf_matrix)

In [6]:
def cal_result2(preds, y):
    conf_matrix = confusion_matrix(y, preds)
    print('혼동행렬')
    print(conf_matrix)
    accuracy = accuracy_score(y, preds)
    print(f'정확도: {accuracy}')
    class_report = classification_report(y, preds)
    print('Classification Report:')
    print(class_report)
    
    return conf_matrix

In [7]:
# 대분류 라벨 생성
def make_large_type_else(df):
    type1 = ['국가사무']
    type2 = ['시도사무', '시군구사무', '시도-시군구 공동사무']
    type3 = ['국가-시도-시군구 공동사무', '국가-시군구공동사무', '국가-시도 공동사무']
    df['사무유형(대분류)'] = 0    # 0, NaN 값들은 0으로 초기화
    
    for name in type1:
        df.loc[(df['사무유형(국가사무, 시도사무, 시군구사무)']==name), '사무유형(대분류)'] = 1
    for name in type2:
        df.loc[(df['사무유형(국가사무, 시도사무, 시군구사무)']==name), '사무유형(대분류)'] = 2
    for name in type3:
        df.loc[(df['사무유형(국가사무, 시도사무, 시군구사무)']==name), '사무유형(대분류)'] = 3
    return df

# 1. 데이터 입력

In [8]:
test_df = pd.read_excel('data/2023 update law_1027_1214_예측대상.xlsx')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 956 entries, 0 to 955
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   부처                           924 non-null    object 
 1   법령명                          924 non-null    object 
 2   법령구분                         924 non-null    float64
 3   조번호                          924 non-null    float64
 4   항번호                          924 non-null    object 
 5   호번호                          924 non-null    float64
 6   조문제목                         924 non-null    object 
 7   조                            924 non-null    object 
 8   조문                           924 non-null    object 
 9   사무판단                         574 non-null    float64
 10  사무판단근거                       286 non-null    object 
 11  사무명                          151 non-null    object 
 12  수행주체(수행기관)(국가, 지방자치단체, 기관)   348 non-null    object 
 13  사무유형(국가사무, 시도사무, 시군구

# 2. 데이터 전처리

In [9]:
test_df = make_large_type_else(test_df)
test_df = f.x_null_drop(test_df)
test_df = f.x_wrong_drop(test_df)
test_df = f.no_work_check(test_df)
test_df = f.make_large_type(test_df)

In [10]:
# 결과 저장 데이터 프레임 생성
check_cols = ['법령명' ,'조문제목', '조문', '사무판단', '사무유형(대분류)']
sub_df = test_df.loc[:, check_cols].copy()
sub_df.shape

(905, 5)

# 3. Rule-Base 적용

In [11]:
sub_df = f.rule_based(sub_df)  # rule_base 조문 확인

# rule_base로 걸러낸 조문들
rule_base_filter_idx = sub_df.loc[sub_df['rule_based']==0, :].index

# 4. 1차 사무판단
## encoder 모델

* encoder 구조 생성

In [12]:
# 모델 및 토크나이저 불러오기
model_name = 'klue/roberta-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 모델 기본 구조 설정
input_size = 512
hidden_size = 256
output_size = 2
num_encoder_layers = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = mm.MyModel(input_size, hidden_size, output_size, num_encoder_layers)
model.to(device)

MyModel(
  (encoder): MyEncoder(
    (layers): ModuleList(
      (0): MyEncoderLayer(
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (feedforward): Sequential(
          (0): Linear(in_features=512, out_features=1024, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1024, out_features=512, bias=True)
        )
        (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (layer_norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (layer1): Linear(in_features=512, out_features=256, bias=True)
  (batch_norm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (layer2): Linear(in_features=256, out_features=2, bias=True)
)

* 학습된 encoder 불러오기

In [13]:
model_path = 'pickles/encoder_model_f1_all.pth'  # Provide the correct path
model.load_state_dict(torch.load(model_path, map_location=device))

<All keys matched successfully>

* 모델 평가

In [14]:
# 데이터로더 생성
batch_size = 16  # 배치크기 조절

test_dataset = CustomDataset1(sub_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [15]:
model.eval()

test_loss = 0
probs = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc=f'Validation'):
        inputs = {key: value.to(device) for key, value in batch.items() if key != 'labels'}
        labels = batch['labels'].to(device)

        # 모델에 토큰화된 입력 데이터 전달
        outputs = model(**inputs).float()
        test_loss += torch.nn.functional.cross_entropy(outputs, labels.long())
        
        # 소프트맥스를 적용하여 확률값 얻기
        probs_batch = torch.nn.functional.softmax(outputs, dim=1)
        probs.append(probs_batch.cpu().numpy())

# 손실 계산
avg_test_loss = test_loss / len(test_loader)

# 확률값을 하나의 배열로 통합
probs = np.concatenate(probs, axis=0)

print(avg_test_loss)

Validation: 100%|██████████████████████████████| 57/57 [00:00<00:00, 154.04it/s]

tensor(0.6586)


In [16]:
# 결과 확률 저장
prob_1 = probs[:, 1]
prob_1 = [round(prob, 5) for prob in prob_1]

In [17]:
sub_df.head(1)

,법령명,조문제목,조문,사무판단,사무유형(대분류),rule_based
0,개인정보 보호법,국가 등의 책무,③ 국가와 지방자치단체는 만 14세 미만 아동이 개인정보 처리가 미치는 영향과 ...,1,3,1


## 1차 사무판단 진행

In [18]:
threshold = 0.251    # test 결과로 확인한 threshold

sub_df['사무판단 예측'] = 0
sub_df['사무판단 예측확률'] = prob_1

# 1차 사무판단
sub_df.loc[sub_df['사무판단 예측확률'] >= threshold, '사무판단 예측'] = 1

sub_df.loc[rule_base_filter_idx, '사무판단 예측'] = 0   # rule-base 결과 적용

In [19]:
#sub_df.loc[(sub_df['사무예측 결과']==0)&(sub_df['사무판단']!=0), :]    # 모델 예측 결과 틀린 부분

In [20]:
# 1차 필터링 결과
cal_result(sub_df['사무판단 예측'], sub_df['사무판단'])

Accuracy: 0.46519337016574586
Precision: 0.39974937343358397
Recall: 0.9845679012345679
F1-Score: 0.5686274509803921
Confusion Matrix:
[[102 479]
 [  5 319]]


## 1차 filtering 결과

In [21]:
# 1차 filter 결과
first_filtered_df = sub_df.loc[sub_df['사무판단 예측']==1, :].copy()
first_filtered_df.head(1)

,법령명,조문제목,조문,사무판단,사무유형(대분류),rule_based,사무판단 예측,사무판단 예측확률
0,개인정보 보호법,국가 등의 책무,③ 국가와 지방자치단체는 만 14세 미만 아동이 개인정보 처리가 미치는 영향과 ...,1,3,1,1,0.71948


# 4. 2차 사무판단 + 사무유형 판단
* 사무로 판단되었을때 실행

## Encoder Model

In [22]:
input_size = 128
hidden_size = 64
output_size = 4

total_encoder = mm.MyModel(input_size, hidden_size, output_size)

In [23]:
model_path2 = 'pickles/encoder_model_f2_all.pth'  # Provide the correct path
total_encoder.load_state_dict(torch.load(model_path2, map_location=device))

<All keys matched successfully>

* 수행주체 추출

In [24]:
# 수행주체 종류 불러오기
with open('pickles/subject_list.pkl', 'rb') as file:
    subject_list = pickle.load(file)

In [25]:
# 수행주체(문자열, 리스트 추가)
first_filtered_df = f.new_subject_make(first_filtered_df, subject_list, tqdm)

Processing rows: 100%|██████████████████████| 798/798 [00:00<00:00, 5606.53it/s]


In [26]:
first_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 798 entries, 0 to 955
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   법령명         798 non-null    object 
 1   조문제목        798 non-null    object 
 2   조문          798 non-null    object 
 3   사무판단        798 non-null    int64  
 4   사무유형(대분류)   798 non-null    int64  
 5   rule_based  798 non-null    int64  
 6   사무판단 예측     798 non-null    int64  
 7   사무판단 예측확률   798 non-null    float32
 8   수행주체(리스트)   798 non-null    object 
 9   수행주체(문자열)   798 non-null    object 
dtypes: float32(1), int64(4), object(5)
memory usage: 97.8+ KB


In [27]:
# 데이터로더 생성
batch_size = 16  # 배치크기 조절

total_dataset = CustomDataset2(first_filtered_df, tokenizer)
total_loader = DataLoader(total_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [28]:
# 2차 인코더 가중치 학습
class_weights = [0.2969,  2.2733,  9.0880, 12.1487]
class_weights = torch.tensor(class_weights, dtype=torch.float32)

In [29]:
total_encoder.eval()
val_loss = 0.0
y_true = []  # 실제 레이블을 저장할 리스트
y_pred = []  # 모델의 예측 결과를 저장할 리스트
y_probs = [] # 사무유형별 확률 저장할 리스트
with torch.no_grad():
    for batch in tqdm(total_loader, desc=f'Test'):
        inputs = {key: value.to(device) for key, value in batch.items() if key != 'labels'}
        labels = batch['labels'].to(device)

        # 모델에 토큰화된 입력 데이터 전달
        outputs = total_encoder(**inputs).float()
        
        probs = torch.nn.functional.softmax(outputs, dim=1)
        y_probs.extend(probs.cpu().numpy())
        _, predictions = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())
        val_loss += torch.nn.functional.cross_entropy(outputs, labels, weight=class_weights)

avg_val_loss = val_loss / len(total_loader)
print(f'Test Loss: {avg_val_loss}')

Test: 100%|████████████████████████████████████| 50/50 [00:00<00:00, 283.56it/s]

Test Loss: 1.3529672622680664


In [30]:
probs = pd.DataFrame(y_probs)
probs = probs.set_index(first_filtered_df.index)

# 인코더 결과 저장
encoder_result_df = f.make_result_df(pd, y_true, y_pred, probs[0], probs[1], probs[2], probs[3])

In [31]:
encoder_result_df

,사무유형(대분류),사무유형(대분류) 예측,비사무 확률,국가 확률,지방 확률,공동 확률
0,3,0,0.34137,0.30984,0.13518,0.21361
1,0,0,0.53890,0.21358,0.08057,0.16695
2,0,0,0.42994,0.24136,0.13480,0.19389
3,0,0,0.50787,0.24565,0.08548,0.16100
4,0,0,0.53112,0.20586,0.08917,0.17385
...,...,...,...,...,...,...
951,2,1,0.25254,0.32641,0.26755,0.15351
952,1,0,0.53202,0.22297,0.14500,0.10001
953,1,1,0.20129,0.32805,0.22061,0.25005
954,1,2,0.24101,0.26328,0.27869,0.21701


In [32]:
# 결과 확인
cal_result2(encoder_result_df['사무유형(대분류) 예측'], encoder_result_df['사무유형(대분류)'])

혼동행렬
[[357  87  24   7]
 [120  73  27   1]
 [ 26  19  16   2]
 [ 12  13  11   3]]
정확도: 0.5626566416040101
Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.75      0.72       475
           1       0.38      0.33      0.35       221
           2       0.21      0.25      0.23        63
           3       0.23      0.08      0.12        39

    accuracy                           0.56       798
   macro avg       0.38      0.35      0.35       798
weighted avg       0.55      0.56      0.55       798



array([[357,  87,  24,   7],
       [120,  73,  27,   1],
       [ 26,  19,  16,   2],
       [ 12,  13,  11,   3]])

## Random Forest

In [33]:
rf_2_filtered_df = first_filtered_df.copy()

In [34]:
# RF 모델 불러오기
with open('pickles/rf_model_f2_all.pkl', 'rb') as file:
    model_2_rf = pickle.load(file)

In [35]:
rf_2_filtered_df.head(1)

,법령명,조문제목,조문,사무판단,사무유형(대분류),rule_based,사무판단 예측,사무판단 예측확률,수행주체(리스트),수행주체(문자열)
0,개인정보 보호법,국가 등의 책무,③ 국가와 지방자치단체는 만 14세 미만 아동이 개인정보 처리가 미치는 영향과 ...,1,3,1,1,0.71948,"[국가, 단체, 지방자치단체]",국가 단체 지방자치단체


In [36]:
# 법령명 통합
rf_2_filtered_df['법령명'] = rf_2_filtered_df['법령명'].apply(lambda x: re.sub(r'\s?시행령|\s?시행규칙', '', x))

In [37]:
# 새로운 컬럼을 이용하기 위한 수행주체 count dictionary
with open('pickles/subject_dic_f2_all.pkl', 'rb') as file:
    subject_dic = pickle.load(file)

In [38]:
# 수행주체 기준 새로운 열 생성
rf_2_filtered_df = f.score_subject(rf_2_filtered_df,subject_dic,tqdm)

Processing rows: 100%|██████████████████████| 798/798 [00:00<00:00, 5855.80it/s]


In [39]:
select_column = ['법령명', 'score_subject_nan', 'score_subject_n', 'score_subject_r', 'score_subject_p']
rf_2_filtered_label = rf_2_filtered_df['사무유형(대분류)']
rf_2_filtered_input = rf_2_filtered_df[select_column]

In [40]:
# '법령명' label encoding 진행
le = LabelEncoder()
rf_2_filtered_input['법령명'] = le.fit_transform(rf_2_filtered_input['법령명']) # train set에는 fit_transform

/var/folders/zr/_f1rgf8n0w3541q9k8p3smhr0000gn/T/ipykernel_69438/253750670.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rf_2_filtered_input['법령명'] = le.fit_transform(rf_2_filtered_input['법령명']) # train set에는 fit_transform


In [41]:
# rf 결과 추출
y_pred_rf = model_2_rf.predict(rf_2_filtered_input)
y_prob_rf = model_2_rf.predict_proba(rf_2_filtered_input)

In [42]:
# rf 결과 데이터프레임 생성
rf_probs = pd.DataFrame(y_prob_rf)
rf_probs = rf_probs.set_index(first_filtered_df.index)

rf_result_df = f.make_result_df(pd, rf_2_filtered_label, y_pred_rf, rf_probs[0], rf_probs[1], rf_probs[2], rf_probs[3])

In [43]:
# rf 결과 확인
cal_result2(rf_result_df['사무유형(대분류) 예측'], rf_result_df['사무유형(대분류)'])

혼동행렬
[[418  31  24   2]
 [167  53   1   0]
 [ 50   0  12   1]
 [ 30   2   0   7]]
정확도: 0.6140350877192983
Classification Report:
              precision    recall  f1-score   support

           0       0.63      0.88      0.73       475
           1       0.62      0.24      0.35       221
           2       0.32      0.19      0.24        63
           3       0.70      0.18      0.29        39

    accuracy                           0.61       798
   macro avg       0.57      0.37      0.40       798
weighted avg       0.60      0.61      0.57       798



array([[418,  31,  24,   2],
       [167,  53,   1,   0],
       [ 50,   0,  12,   1],
       [ 30,   2,   0,   7]])

## Ensemble

In [44]:
pred = [0]*len(y_prob_rf)
e_prob0 = (rf_result_df['비사무 확률'] + encoder_result_df['비사무 확률'])/2
e_prob1 = (rf_result_df['국가 확률'] + encoder_result_df['국가 확률'])/2
e_prob2 = (rf_result_df['지방 확률'] + encoder_result_df['지방 확률'])/2
e_prob3 = (rf_result_df['공동 확률'] + encoder_result_df['공동 확률'])/2

ensemble_result_df_11 = f.make_result_df(pd, rf_2_filtered_label, pred, e_prob0, e_prob1, e_prob2, e_prob3)
ensemble_result_df_11['수행주체(리스트)'] = rf_2_filtered_df['수행주체(리스트)']

In [45]:
# 앙상블 사무유형 판단 저장
ensemble_result_df_11 = f.make_ensemble_pred(ensemble_result_df_11)
ensemble_result_df_11['사무유형(대분류) 예측'] = ensemble_result_df_11['사무유형(대분류) 예측'].astype(int)
ensemble_result_df_11['rule_based'] = first_filtered_df['rule_based']

In [46]:
# 앙살블 결과 확인
cal_result2(ensemble_result_df_11['사무유형(대분류) 예측'], ensemble_result_df_11['사무유형(대분류)'])

혼동행렬
[[437  25  12   1]
 [165  56   0   0]
 [ 51   0  11   1]
 [ 30   3   1   5]]
정확도: 0.6378446115288221
Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.92      0.75       475
           1       0.67      0.25      0.37       221
           2       0.46      0.17      0.25        63
           3       0.71      0.13      0.22        39

    accuracy                           0.64       798
   macro avg       0.62      0.37      0.40       798
weighted avg       0.64      0.64      0.58       798



array([[437,  25,  12,   1],
       [165,  56,   0,   0],
       [ 51,   0,  11,   1],
       [ 30,   3,   1,   5]])

In [47]:
# 확실/주의 나누기 [확실: need_to_check = 0 / 주의: need_to_check = 1]
ensemble_result_df_11, good_df, bad_df = f.make_need_to_check(ensemble_result_df_11,
                                                           rf_result_df, encoder_result_df)

In [48]:
ensemble_result_df_11.head(1)

,사무유형(대분류),사무유형(대분류) 예측,비사무 확률,국가 확률,지방 확률,공동 확률,수행주체(리스트),rule_based,need_to_check
0,3,0,0.62209,0.15492,0.06759,0.1554,"[국가, 단체, 지방자치단체]",1,0


In [49]:
# 확실 결과
cal_result2(good_df['사무유형(대분류) 예측'].tolist(), good_df['사무유형(대분류)'].tolist())

혼동행렬
[[310   8   0   0]
 [ 91  18   0   0]
 [ 20   0   4   0]
 [  9   1   0   1]]
정확도: 0.7207792207792207
Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.97      0.83       318
           1       0.67      0.17      0.26       109
           2       1.00      0.17      0.29        24
           3       1.00      0.09      0.17        11

    accuracy                           0.72       462
   macro avg       0.85      0.35      0.39       462
weighted avg       0.73      0.72      0.65       462



array([[310,   8,   0,   0],
       [ 91,  18,   0,   0],
       [ 20,   0,   4,   0],
       [  9,   1,   0,   1]])

In [50]:
# 애매 결과
cal_result2(bad_df['사무유형(대분류) 예측'].tolist(), bad_df['사무유형(대분류)'].tolist())

혼동행렬
[[127  17  12   1]
 [ 74  38   0   0]
 [ 31   0   7   1]
 [ 21   2   1   4]]
정확도: 0.5238095238095238
Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.81      0.62       157
           1       0.67      0.34      0.45       112
           2       0.35      0.18      0.24        39
           3       0.67      0.14      0.24        28

    accuracy                           0.52       336
   macro avg       0.55      0.37      0.39       336
weighted avg       0.55      0.52      0.49       336



array([[127,  17,  12,   1],
       [ 74,  38,   0,   0],
       [ 31,   0,   7,   1],
       [ 21,   2,   1,   4]])

## 2차 filter 결과 저장

In [51]:
sencond_filtered_df = ensemble_result_df_11.copy()   # 2차 필터 결과 저장

In [52]:
# 2차 filter 결과 확인
cal_result2(sencond_filtered_df['사무유형(대분류) 예측'], sencond_filtered_df['사무유형(대분류)'])

혼동행렬
[[437  25  12   1]
 [165  56   0   0]
 [ 51   0  11   1]
 [ 30   3   1   5]]
정확도: 0.6378446115288221
Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.92      0.75       475
           1       0.67      0.25      0.37       221
           2       0.46      0.17      0.25        63
           3       0.71      0.13      0.22        39

    accuracy                           0.64       798
   macro avg       0.62      0.37      0.40       798
weighted avg       0.64      0.64      0.58       798



array([[437,  25,  12,   1],
       [165,  56,   0,   0],
       [ 51,   0,  11,   1],
       [ 30,   3,   1,   5]])

In [53]:
# 2차 filter (확실/주의)결과 확인
h_ratio, j_ratio = f.check_result(sencond_filtered_df)

주의 사무 개수: 336
주의에서 틀린 개수: 160
주의에서 맞은 개수: 176
------------------
확실 사무 개수: 462
확실에서 틀린 개수: 129
확실에서 맞은 개수: 333
확실 정확도: 0.7207792207792207
애매 정확도: 0.5238095238095238


In [54]:
sencond_filtered_df.head(1)

,사무유형(대분류),사무유형(대분류) 예측,비사무 확률,국가 확률,지방 확률,공동 확률,수행주체(리스트),rule_based,need_to_check
0,3,0,0.62209,0.15492,0.06759,0.1554,"[국가, 단체, 지방자치단체]",1,0


In [55]:
sub_df.head(1)

,법령명,조문제목,조문,사무판단,사무유형(대분류),rule_based,사무판단 예측,사무판단 예측확률
0,개인정보 보호법,국가 등의 책무,③ 국가와 지방자치단체는 만 14세 미만 아동이 개인정보 처리가 미치는 영향과 ...,1,3,1,1,0.71948


In [56]:
# 최종 결과 저장
sub_df = f.make_final_df(sub_df, sencond_filtered_df)
sub_df

,법령명,조문제목,조문,사무판단,사무유형(대분류),rule_based,사무판단 예측,사무판단 예측확률,수행주체,사무유형(대분류) 결과,비사무 확률,국가 확률,지방 확률,공동 확률,need_to_check
0,개인정보 보호법,국가 등의 책무,③ 국가와 지방자치단체는 만 14세 미만 아동이 개인정보 처리가 미치는 영향과 ...,1,3,1.0,1,0.71948,"[국가, 단체, 지방자치단체]",0,0.62209,0.15492,0.06759,0.15540,0
1,개인정보 보호법,다른 법률과의 관계,② 개인정보의 처리 및 보호에 관한 다른 법률을 제정하거나 개정하는 경우에는 이...,0,0,1.0,1,0.59204,[],0,0.76945,0.10679,0.04028,0.08348,0
2,개인정보 보호법,민감정보의 처리 제한,③ 개인정보처리자는 재화 또는 서비스를 제공하는 과정에서 공개되는 정보에 정보주...,0,0,1.0,1,0.80474,[],0,0.71497,0.12068,0.06740,0.09695,0
3,개인정보 보호법,업무위탁에 따른 개인정보의 처리 제한,⑥ 수탁자는 위탁받은 개인정보의 처리 업무를 제3자에게 다시 위탁하려는 경우에는...,0,0,1.0,1,0.61685,[],0,0.75394,0.12282,0.04274,0.08050,0
4,개인정보 보호법,가명정보에 대한 안전조치의무 등,② 개인정보처리자는 제28조의2 또는 제28조의3에 따라 가명정보를 처리하는 경...,0,0,1.0,1,0.69162,[],0,0.76556,0.10293,0.04459,0.08692,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951,하천법,하천의 구분 및 지정,⑧ 시ㆍ도지사는 지방하천이 제2항 각 호의 어느 하나에 해당한다고 판단하는 경우...,1,2,1.0,1,0.48119,"[장관, 관리, 국가, 도지사, 위원회, 시ㆍ도, 시ㆍ도지사, 환경부]",0,0.42670,0.25820,0.14877,0.16633,1
952,하천법,하천관리 자료의 정보화,② 환경부장관은 제1항에 따른 하천관리 정보체계를 구축할 경우 최신의 정보통신기...,1,1,1.0,1,0.69429,"[장관, 관리, 환경부]",1,0.32187,0.55562,0.07250,0.05001,1
953,하천법,하천관리청의 하천공사 및 유지ㆍ보수,⑤ 환경부장관은 국가하천 배수영향구간을 고시하려는 경우에는 관계 중앙행정기관의 ...,1,1,1.0,1,0.64288,"[기관, 장관, 중앙행정기관, 국가, 행정기관, 환경부]",0,0.46624,0.29843,0.11030,0.12503,1
954,하천법,하천관리청의 하천공사 및 유지ㆍ보수,⑪ 환경부장관은 제6항에 따라 국가하천의 유지ㆍ보수를 시행하고자 하는 때에는 미...,1,1,1.0,1,0.68251,"[장관, 국가, 도지사, 시ㆍ도, 시ㆍ도지사, 환경부]",0,0.46050,0.18164,0.23935,0.11851,1


In [58]:
cal_result2(sub_df['사무유형(대분류) 결과'], sub_df['사무유형(대분류)'])

혼동행렬
[[538  25  12   1]
 [165  56   0   0]
 [ 56   0  11   1]
 [ 31   3   1   5]]
정확도: 0.6740331491712708
Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.93      0.79       576
           1       0.67      0.25      0.37       221
           2       0.46      0.16      0.24        68
           3       0.71      0.12      0.21        40

    accuracy                           0.67       905
   macro avg       0.63      0.37      0.40       905
weighted avg       0.66      0.67      0.62       905



array([[538,  25,  12,   1],
       [165,  56,   0,   0],
       [ 56,   0,  11,   1],
       [ 31,   3,   1,   5]])

top2 뽑기

In [59]:
ex = sub_df.copy()
ex.head()

,법령명,조문제목,조문,사무판단,사무유형(대분류),rule_based,사무판단 예측,사무판단 예측확률,수행주체,사무유형(대분류) 결과,비사무 확률,국가 확률,지방 확률,공동 확률,need_to_check
0,개인정보 보호법,국가 등의 책무,③ 국가와 지방자치단체는 만 14세 미만 아동이 개인정보 처리가 미치는 영향과 ...,1,3,1.0,1,0.71948,"[국가, 단체, 지방자치단체]",0,0.62209,0.15492,0.06759,0.15540,0
1,개인정보 보호법,다른 법률과의 관계,② 개인정보의 처리 및 보호에 관한 다른 법률을 제정하거나 개정하는 경우에는 이...,0,0,1.0,1,0.59204,[],0,0.76945,0.10679,0.04028,0.08348,0
2,개인정보 보호법,민감정보의 처리 제한,③ 개인정보처리자는 재화 또는 서비스를 제공하는 과정에서 공개되는 정보에 정보주...,0,0,1.0,1,0.80474,[],0,0.71497,0.12068,0.06740,0.09695,0
3,개인정보 보호법,업무위탁에 따른 개인정보의 처리 제한,⑥ 수탁자는 위탁받은 개인정보의 처리 업무를 제3자에게 다시 위탁하려는 경우에는...,0,0,1.0,1,0.61685,[],0,0.75394,0.12282,0.04274,0.08050,0
4,개인정보 보호법,가명정보에 대한 안전조치의무 등,② 개인정보처리자는 제28조의2 또는 제28조의3에 따라 가명정보를 처리하는 경...,0,0,1.0,1,0.69162,[],0,0.76556,0.10293,0.04459,0.08692,0


In [66]:
ex['top2 결과'] = 0
for i in ex.index:
    l = list(ex.loc[i, '비사무 확률':'공동 확률'])
    #print(l)
    s = set(l)
    if len(s) == 1:
        continue
    sorted_lst = sorted(l)
    second_largest = sorted_lst[-2]
    second_largest_index = l.index(second_largest)
    ex.loc[i, 'top2 결과'] = second_largest_index

In [67]:
ex['top2 정답'] = 0
idxs = (ex['사무유형(대분류)']==ex['사무유형(대분류) 결과'])|(ex['사무유형(대분류)']==ex['top2 결과'])
ex.loc[idxs ,'top2 정답'] = ex.loc[idxs ,'사무유형(대분류)']

cal_result2(ex['top2 정답'], ex['사무유형(대분류)'])

혼동행렬
[[576   0   0   0]
 [ 16 205   0   0]
 [ 21   0  47   0]
 [ 25   0   0  15]]
정확도: 0.9314917127071823
Classification Report:
              precision    recall  f1-score   support

           0       0.90      1.00      0.95       576
           1       1.00      0.93      0.96       221
           2       1.00      0.69      0.82        68
           3       1.00      0.38      0.55        40

    accuracy                           0.93       905
   macro avg       0.98      0.75      0.82       905
weighted avg       0.94      0.93      0.92       905



array([[576,   0,   0,   0],
       [ 16, 205,   0,   0],
       [ 21,   0,  47,   0],
       [ 25,   0,   0,  15]])

* 정오표

In [69]:
# 정오표 생성
sub_df['사무판단_정오'] = 0
sub_df.loc[(sub_df['사무판단']==0)&(sub_df['사무판단']==ex['사무유형(대분류) 결과']), '사무판단_정오'] = 0
sub_df.loc[(sub_df['사무판단']==0)&(sub_df['사무판단']!=ex['사무유형(대분류) 결과']), '사무판단_정오'] = 1
sub_df.loc[(sub_df['사무판단']==1)&(sub_df['사무판단']!=ex['사무유형(대분류) 결과']), '사무판단_정오'] = 2
sub_df.loc[(sub_df['사무판단']==1)&(sub_df['사무판단']==ex['사무유형(대분류) 결과']), '사무판단_정오'] = 3

## 최종 데이터 저장

In [81]:
a = test_df.loc[:, :'수탁기관']
b = sub_df.loc[:, '사무유형(대분류)':]

In [83]:
result = pd.concat([a, b], axis=1)
result.to_csv('data/final_df(test).csv', index=False)